In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

In [33]:
imdb=keras.datasets.imdb
vocabulary_size = 1000
Max_len = 500
(train_sequences, train_labels), (test_sequences, test_labels) = tf.keras.datasets.imdb.load_data(num_words=vocabulary_size)
(train_sequences_removed, train_labels_removed), (test_sequences_removed, test_labels_removed) = tf.keras.datasets.imdb.load_data(num_words=vocabulary_size, maxlen=(Max_len+1))
print('Loaded dataset with {} training samples, {} test samples'.format(len(X_train), len(X_test)))

Loaded dataset with 25000 training samples, 25000 test samples


In [5]:
print (train_sequences[6]) # numbers
print(test_sequences[6]) #label

[1, 2, 365, 2, 5, 2, 354, 11, 14, 2, 2, 7, 2, 2, 2, 356, 44, 4, 2, 500, 746, 5, 200, 4, 2, 11, 2, 2, 2, 2, 2, 5, 2, 26, 6, 2, 2, 17, 369, 37, 215, 2, 143, 2, 5, 2, 8, 2, 15, 36, 119, 257, 85, 52, 486, 9, 6, 2, 2, 63, 271, 6, 196, 96, 949, 2, 4, 2, 7, 4, 2, 2, 819, 63, 47, 77, 2, 180, 6, 227, 11, 94, 2, 2, 13, 423, 4, 168, 7, 4, 22, 5, 89, 665, 71, 270, 56, 5, 13, 197, 12, 161, 2, 99, 76, 23, 2, 7, 419, 665, 40, 91, 85, 108, 7, 4, 2, 5, 2, 81, 55, 52, 2]
1


In [36]:
word_ind = imdb.get_word_index()
id2word = {i: word for word, i in word_ind.items()}
print('---review with words---')
print([id2word.get(i, ' ') for i in train_sequences[6]])
print('---label---')
print(train_labels[6])

---review with words---
['the', 'and', 'full', 'and', 'to', 'and', 'boring', 'this', 'as', 'and', 'and', 'br', 'and', 'and', 'and', 'need', 'has', 'of', 'and', 'b', 'message', 'to', 'may', 'of', 'and', 'this', 'and', 'and', 'and', 'and', 'and', 'to', 'and', 'he', 'is', 'and', 'and', 'movie', 'women', 'like', "isn't", 'and', "i'm", 'and', 'to', 'and', 'in', 'and', 'for', 'from', 'did', 'having', 'because', 'very', 'quality', 'it', 'is', 'and', 'and', 'really', 'book', 'is', 'both', 'too', 'worked', 'and', 'of', 'and', 'br', 'of', 'and', 'and', 'figure', 'really', 'there', 'will', 'and', 'things', 'is', 'far', 'this', 'make', 'and', 'and', 'was', "couldn't", 'of', 'few', 'br', 'of', 'you', 'to', "don't", 'female', 'than', 'place', 'she', 'to', 'was', 'between', 'that', 'nothing', 'and', 'movies', 'get', 'are', 'and', 'br', 'yes', 'female', 'just', 'its', 'because', 'many', 'br', 'of', 'and', 'to', 'and', 'people', 'time', 'very', 'and']
---label---
1


In [25]:
#def one_hot_encode(data): 
 # for i in X_train:
  #  data = tf.one_hot(indices=i,depth=vocabulary_size)
    
  #return data;

In [37]:
sequence_lengths = [len(sequence) for sequence in train_sequences]
max_len = max(sequence_lengths)
min_len = min(sequence_lengths)
print(len(sequence_lengths))
print(max_len)
print(min_len)

25000
2494
11


In [38]:
seq_len = 200
train_sequences_removed_padded = tf.keras.preprocessing.sequence.pad_sequences(train_sequences_removed, maxlen=Max_len)
test_sequences_removed_padded = tf.keras.preprocessing.sequence.pad_sequences(test_sequences_removed, maxlen=Max_len)

train_data_removed = tf.data.Dataset.from_tensor_slices((train_sequences_removed_padded, train_labels_removed.astype(np.float32))).batch(batch_size, drop_remainder = True).repeat()
test_data_removed = tf.data.Dataset.from_tensor_slices((train_sequences_removed_padded, train_labels_removed.astype(np.float32))).batch(batch_size, drop_remainder=True)

sequence_lengths_removed = [len(sequence) for sequence in train_sequences_removed_padded]
max_len_rem = max(sequence_lengths_removed)
min_len_rem = min(sequence_lengths_removed)
print(len(sequence_lengths_removed))
print(max_len_rem)
print(min_len_rem)

22892
500
500


In [26]:
optimizer = tf.optimizers.Adam()

loss_fn = tf.losses.BinaryCrossentropy(from_logits=True)

train_acc_metric = tf.metrics.BinaryAccuracy

training_steps = 500

In [27]:
def rnn_model(sequences):
  old_state = tf.zeros((128, 50))
  for step in range(500):
    x_t = sequences[:, step]
    x_t = tf.one_hot(x_t, depth = vocabulary_size)
    a_t = tf.math.add(bias, tf.matmul(old_state, W), tf.matmul(x_t, U))
    new_state = tf.math.tanh(a_t)
    old_state = new_state

  out = output_layer(old_state)
  return out

def output_layer(out):
  o_t =tf.math.add(out_bias, tf.matmul(out, V)) 
  op = tf.keras.activations.sigmoid(o_t)

  return op

In [28]:
def train_step( sequences, labels):
    with tf.GradientTape() as tape:
        logits = rnn_model(sequences)
        xent = loss_fn(labels, logits)

    variables = [U, W, V, bias, out_bias]
    gradients = tape.gradient(xent, variables)
    optimizer.apply_gradients((gradients,variables) for(gradients, variables) in zip(gradients, variables) if gradients is not None)

    return xent, logits

In [41]:
U = tf.Variable(np.random.uniform(low=-0.1, high=0.1, size=[1000, 50]).astype(np.float32), name="U")
W = tf.Variable(np.random.uniform(low=-0.1, high=0.1, size=[50, 50]).astype(np.float32), name="W")
bias = tf.Variable(np.random.uniform(low=-0.1, high=0.1, size=[128, 50]).astype(np.float32), name="bias")
V = tf.Variable(np.random.uniform(low=-0.1, high=0.1, size=[50, 1]).astype(np.float32), name="V")
out_bias = tf.Variable(np.random.uniform(low=-0.1, high=0.1, size=[128, 1]).astype(np.float32), name="out_bias")
for step, (sequence_batch, label_batch) in enumerate (train_data_removed):
    if step > training_steps:
        break
    xent, logits = train_step(sequence_batch, label_batch)
    if not step % 100:
        train_acc_metric = tf.metrics.binary_accuracy(label_batch, logits)
        acc = tf.math.reduce_mean(train_acc_metric)
        print(" Loss: {} Accuracy: {}".format(step, xent, acc))

 Loss: 0 Accuracy: 0.7628380060195923
 Loss: 100 Accuracy: 0.7036460638046265
 Loss: 200 Accuracy: 0.7054107785224915
 Loss: 300 Accuracy: 0.6909956336021423


In [ ]:
for sequence_batch, label_batch in test_data_removed:

    xent, logits = train_step(sequence_batch, label_batch)
    train_acc_metric = tf.metrics.binary_accuracy(label_batch, logits)
    acc = tf.math.reduce_mean(train_acc_metric)
    print(" Loss: {} Accuracy: {}".format(step, xent, acc))
    break

1) Food for thought #1: Why is this wasteful? 

Important information might be eliminated due to elimination of longer sequence.
smarter way-

2) Food for thought #2: Between truncating long sequences and removing them, which option do you think is better? Why?

Removing might be better.Because with truncate we truncate the entire sequence,
with remove we remove only a part.


3)Food for thought #3: Can you think of a way to avoid the one-hot vectors completely? Even if you cannot implement it, a conceptual idea is fine.

Dummy encoding- uses N-1 features to represent N labels/categories.
LabelEncoder

5)Food for thought #5: All sequences start with the same special “beginning of sequence” token (coded by index 1). Given this fact, is there a point in learning an initial state? Why (not)?

Yes we need to learn. As it contributes in the next state calculation.

6)pad_sequences allows for pre or post padding. Try both to see the difference. Which option do you think is better?

Pre-padding is better. 
- Noises can be less
- Due to post padding of zeros the final output might get flushed out.